# 基础股票回测示例

这个笔记本演示如何使用Djinn框架进行基础的股票回测。

In [ ]:
# 导入必要的库
import sys
import os
sys.path.insert(0, os.path.abspath('../../'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置绘图样式
sns.set_style("whitegrid")
plt.style.use('seaborn-v0_8')

print("环境设置完成")

## 1. 导入Djinn框架

In [ ]:
from djinn.data.fetcher import DataFetcher
from djinn.data.providers.yahoo_finance import YahooFinanceProvider
from djinn.core.strategy.base import Strategy
from djinn.core.strategy.indicators import TechnicalIndicators
from djinn.core.backtest.event_driven import EventDrivenBacktestEngine
from djinn.core.backtest.vectorized import VectorizedBacktestEngine
from djinn.utils.logger import get_logger

# 设置日志
logger = get_logger(__name__)
print("Djinn框架导入完成")

## 2. 获取数据

In [ ]:
# 创建数据获取器
provider = YahooFinanceProvider()
fetcher = DataFetcher(provider)

# 定义要获取的股票
symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

# 定义时间范围
end_date = datetime.now()
start_date = end_date - timedelta(days=365*2)  # 2年数据

print(f"获取数据: {symbols}")
print(f"时间范围: {start_date.date()} 到 {end_date.date()}")

In [ ]:
# 获取数据
data = {}
for symbol in symbols:
    try:
        df = fetcher.fetch_historical_data(
            symbol=symbol,
            start_date=start_date,
            end_date=end_date,
            interval='1d'
        )
        if df is not None and not df.empty:
            data[symbol] = df
            print(f"{symbol}: 获取到 {len(df)} 条数据")
        else:
            print(f"{symbol}: 未获取到数据")
    except Exception as e:
        print(f"{symbol}: 获取数据失败 - {e}")

print(f"\n成功获取 {len(data)} 只股票的数据")

## 3. 查看数据

In [ ]:
# 查看第一只股票的数据
if data:
    first_symbol = list(data.keys())[0]
    df = data[first_symbol]
    
    print(f"{first_symbol} 数据概览:")
    print(f"数据形状: {df.shape}")
    print(f"\n前5行数据:")
    print(df.head())
    print(f"\n后5行数据:")
    print(df.tail())
    print(f"\n数据统计:")
    print(df.describe())

In [ ]:
# 绘制价格走势
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, (symbol, df) in enumerate(data.items()):
    if i >= len(axes):
        break
        
    ax = axes[i]
    ax.plot(df.index, df['close'], label='收盘价', linewidth=2)
    ax.set_title(f'{symbol} 价格走势')
    ax.set_xlabel('日期')
    ax.set_ylabel('价格 (USD)')
    ax.legend()
    ax.grid(True, alpha=0.3)

# 隐藏多余的子图
for i in range(len(data), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

## 4. 创建简单策略

In [ ]:
class SimpleMovingAverageStrategy(Strategy):
    """简单的移动平均线交叉策略"""
    
    def __init__(self, fast_period=20, slow_period=50):
        super().__init__(name="简单移动平均策略")
        self.fast_period = fast_period
        self.slow_period = slow_period
        self.min_data_points = max(fast_period, slow_period) + 10
        
    def initialize(self, data: dict):
        """初始化策略"""
        print(f"策略初始化: {self.name}")
        print(f"快速均线周期: {self.fast_period}")
        print(f"慢速均线周期: {self.slow_period}")
        
    def calculate_signal(self, symbol: str, data: pd.DataFrame, current_date: datetime) -> float:
        """计算交易信号"""
        if len(data) < self.min_data_points:
            return 0.0
        
        # 计算移动平均线
        close_prices = data['close']
        
        fast_ma = close_prices.rolling(window=self.fast_period).mean()
        slow_ma = close_prices.rolling(window=self.slow_period).mean()
        
        # 获取最新值
        current_fast = fast_ma.iloc[-1]
        current_slow = slow_ma.iloc[-1]
        current_price = close_prices.iloc[-1]
        
        # 生成信号
        signal = 0.0
        
        # 金叉: 快线上穿慢线
        if current_fast > current_slow and fast_ma.iloc[-2] <= slow_ma.iloc[-2]:
            signal = 1.0  # 强烈买入信号
        # 死叉: 快线下穿慢线
        elif current_fast < current_slow and fast_ma.iloc[-2] >= slow_ma.iloc[-2]:
            signal = -1.0  # 强烈卖出信号
        # 趋势跟踪
        elif current_price > current_fast > current_slow:
            signal = 0.5  # 温和买入信号
        elif current_price < current_fast < current_slow:
            signal = -0.5  # 温和卖出信号
            
        return signal
    
    def calculate_signals_vectorized(self, data: pd.DataFrame) -> pd.Series:
        """向量化信号计算"""
        close_prices = data['close']
        
        # 计算移动平均线
        fast_ma = close_prices.rolling(window=self.fast_period).mean()
        slow_ma = close_prices.rolling(window=self.slow_period).mean()
        
        # 初始化信号序列
        signals = pd.Series(0.0, index=close_prices.index)
        
        # 金叉信号
        golden_cross = (fast_ma > slow_ma) & (fast_ma.shift(1) <= slow_ma.shift(1))
        signals[golden_cross] = 1.0
        
        # 死叉信号
        death_cross = (fast_ma < slow_ma) & (fast_ma.shift(1) >= slow_ma.shift(1))
        signals[death_cross] = -1.0
        
        # 趋势信号
        uptrend = (close_prices > fast_ma) & (fast_ma > slow_ma) & ~golden_cross
        signals[uptrend] = 0.5
        
        downtrend = (close_prices < fast_ma) & (fast_ma < slow_ma) & ~death_cross
        signals[downtrend] = -0.5
        
        return signals

In [ ]:
# 创建策略实例
strategy = SimpleMovingAverageStrategy(fast_period=20, slow_period=50)
print(f"策略创建完成: {strategy.name}")

## 5. 运行回测

In [ ]:
# 创建回测引擎
backtest_engine = EventDrivenBacktestEngine(
    initial_capital=100000.0,
    commission=0.001,  # 0.1% 手续费
    slippage=0.0005,   # 0.05% 滑点
    allow_short=False,
    max_position_size=0.2,  # 最大仓位20%
    stop_loss=0.05,    # 5% 止损
    take_profit=0.10   # 10% 止盈
)

print("回测引擎创建完成")

In [ ]:
# 运行回测
print("开始运行回测...")

# 调整回测时间范围（确保有足够数据）
backtest_start = start_date + timedelta(days=100)  # 跳过前100天用于计算指标
backtest_end = end_date - timedelta(days=10)       # 留出一些缓冲

result = backtest_engine.run(
    strategy=strategy,
    data=data,
    start_date=backtest_start,
    end_date=backtest_end,
    frequency='daily'
)

print("回测完成!")

## 6. 分析回测结果

In [ ]:
# 显示基本性能指标
print("=" * 60)
print("回测结果概览")
print("=" * 60)

print(f"策略名称: {result.strategy_name}")
print(f"回测期间: {result.start_date.strftime('%Y-%m-%d')} 到 {result.end_date.strftime('%Y-%m-%d')}")
print(f"初始资金: ${result.initial_capital:,.2f}")
print(f"最终资金: ${result.final_capital:,.2f}")
print(f"总收益率: {result.total_return:.2%}")
print(f"年化收益率: {result.annual_return:.2%}")
print(f"夏普比率: {result.sharpe_ratio:.2f}")
print(f"最大回撤: {result.max_drawdown:.2%}")
print(f"波动率: {result.volatility:.2%}")
print(f"胜率: {result.win_rate:.2%}")
print(f"总交易次数: {result.total_trades}")
print(f"盈利交易: {result.winning_trades}")
print(f"亏损交易: {result.losing_trades}")
print(f"平均交易收益率: {result.avg_trade_return:.2%}")

In [ ]:
# 将结果转换为DataFrame以便更好查看
result_df = result.to_dataframe()
print("\n详细性能指标:")
print(result_df.to_string(index=False))

In [ ]:
# 绘制权益曲线
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 权益曲线
ax1 = axes[0, 0]
ax1.plot(result.equity_curve.index, result.equity_curve.values, linewidth=2, color='blue')
ax1.set_title('投资组合权益曲线', fontsize=14, fontweight='bold')
ax1.set_xlabel('日期')
ax1.set_ylabel('投资组合价值 (USD)')
ax1.grid(True, alpha=0.3)
ax1.fill_between(result.equity_curve.index, result.equity_curve.values, 
                 result.initial_capital, where=(result.equity_curve.values >= result.initial_capital), 
                 color='green', alpha=0.3, label='盈利')
ax1.fill_between(result.equity_curve.index, result.equity_curve.values, 
                 result.initial_capital, where=(result.equity_curve.values < result.initial_capital), 
                 color='red', alpha=0.3, label='亏损')
ax1.legend()

# 回撤曲线
ax2 = axes[0, 1]
ax2.fill_between(result.drawdown.index, 0, result.drawdown.values * 100, 
                 color='red', alpha=0.5)
ax2.set_title('投资组合回撤', fontsize=14, fontweight='bold')
ax2.set_xlabel('日期')
ax2.set_ylabel('回撤 (%)')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, result.drawdown.max() * 100 * 1.1)

# 月度收益率热图
ax3 = axes[1, 0]
monthly_returns = result.returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
monthly_returns_df = monthly_returns.unstack()

if len(monthly_returns) > 0:
    months = monthly_returns.index.strftime('%Y-%m')
    ax3.bar(range(len(monthly_returns)), monthly_returns.values * 100, 
            color=['green' if x >= 0 else 'red' for x in monthly_returns.values])
    ax3.set_title('月度收益率', fontsize=14, fontweight='bold')
    ax3.set_xlabel('月份')
    ax3.set_ylabel('收益率 (%)')
    ax3.set_xticks(range(len(months)))
    ax3.set_xticklabels(months, rotation=45, ha='right')
    ax3.grid(True, alpha=0.3, axis='y')
else:
    ax3.text(0.5, 0.5, '无足够月度数据', ha='center', va='center', transform=ax3.transAxes)
    ax3.set_title('月度收益率', fontsize=14, fontweight='bold')

# 交易分析
ax4 = axes[1, 1]
if result.trades:
    trade_returns = []
    for trade in result.trades:
        # 简化计算交易收益
        if trade.side == 'buy':
            trade_returns.append(0.01)  # 示例值
        else:
            trade_returns.append(-0.005)  # 示例值
    
    ax4.hist(trade_returns, bins=20, edgecolor='black', alpha=0.7)
    ax4.set_title('交易收益率分布', fontsize=14, fontweight='bold')
    ax4.set_xlabel('交易收益率')
    ax4.set_ylabel('交易次数')
    ax4.grid(True, alpha=0.3, axis='y')
else:
    ax4.text(0.5, 0.5, '无交易数据', ha='center', va='center', transform=ax4.transAxes)
    ax4.set_title('交易收益率分布', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

## 7. 技术指标分析

In [ ]:
# 对第一只股票进行技术指标分析
if data:
    symbol = list(data.keys())[0]
    df = data[symbol]
    
    # 计算技术指标
    indicators = TechnicalIndicators.calculate_all_indicators(df)
    
    print(f"{symbol} 技术指标分析:")
    print("=" * 40)
    
    for name, indicator_result in indicators.items():
        print(f"{name}:")
        if isinstance(indicator_result.values, pd.Series):
            print(f"  最新值: {indicator_result.values.iloc[-1]:.4f}")
        elif isinstance(indicator_result.values, pd.DataFrame):
            print(f"  包含 {len(indicator_result.values.columns)} 个分量")
        if indicator_result.signals is not None:
            latest_signal = indicator_result.signals.iloc[-1]
            signal_text = "买入" if latest_signal > 0 else "卖出" if latest_signal < 0 else "中性"
            print(f"  最新信号: {signal_text} ({latest_signal})")

In [ ]:
# 绘制技术指标图表
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

# 1. 价格与移动平均线
ax1 = axes[0]
ax1.plot(df.index, df['close'], label='收盘价', linewidth=2, color='black', alpha=0.7)

sma_20 = indicators['sma_20'].values
sma_50 = indicators['sma_50'].values

ax1.plot(df.index, sma_20, label='20日SMA', linewidth=1.5, color='blue', alpha=0.8)
ax1.plot(df.index, sma_50, label='50日SMA', linewidth=1.5, color='red', alpha=0.8)

ax1.set_title(f'{symbol} - 价格与移动平均线', fontsize=12, fontweight='bold')
ax1.set_xlabel('日期')
ax1.set_ylabel('价格 (USD)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. RSI指标
ax2 = axes[1]
rsi = indicators['rsi'].values
ax2.plot(df.index, rsi, label='RSI', linewidth=2, color='purple')
ax2.axhline(y=70, color='red', linestyle='--', alpha=0.5, label='超买线 (70)')
ax2.axhline(y=30, color='green', linestyle='--', alpha=0.5, label='超卖线 (30)')
ax2.fill_between(df.index, 70, 100, color='red', alpha=0.1)
ax2.fill_between(df.index, 0, 30, color='green', alpha=0.1)
ax2.set_title('RSI指标', fontsize=12, fontweight='bold')
ax2.set_xlabel('日期')
ax2.set_ylabel('RSI值')
ax2.set_ylim(0, 100)
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. MACD指标
ax3 = axes[2]
macd_df = indicators['macd'].values
ax3.plot(df.index, macd_df['macd'], label='MACD线', linewidth=1.5, color='blue')
ax3.plot(df.index, macd_df['signal'], label='信号线', linewidth=1.5, color='red')
ax3.bar(df.index, macd_df['histogram'], label='柱状图', color=['green' if x >= 0 else 'red' for x in macd_df['histogram']], 
        alpha=0.5, width=1)
ax3.axhline(y=0, color='black', linestyle='-', alpha=0.3)
ax3.set_title('MACD指标', fontsize=12, fontweight='bold')
ax3.set_xlabel('日期')
ax3.set_ylabel('MACD值')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. 布林带
ax4 = axes[3]
bollinger_df = indicators['bollinger'].values
ax4.plot(df.index, df['close'], label='收盘价', linewidth=1.5, color='black', alpha=0.7)
ax4.plot(df.index, bollinger_df['middle'], label='中轨', linewidth=1, color='blue', alpha=0.7)
ax4.plot(df.index, bollinger_df['upper'], label='上轨', linewidth=1, color='red', alpha=0.7, linestyle='--')
ax4.plot(df.index, bollinger_df['lower'], label='下轨', linewidth=1, color='green', alpha=0.7, linestyle='--')
ax4.fill_between(df.index, bollinger_df['upper'], bollinger_df['lower'], color='gray', alpha=0.1)
ax4.set_title('布林带', fontsize=12, fontweight='bold')
ax4.set_xlabel('日期')
ax4.set_ylabel('价格 (USD)')
ax4.legend()
ax4.grid(True, alpha=0.3)

# 5. 成交量
ax5 = axes[4]
ax5.bar(df.index, df['volume'], color='blue', alpha=0.5, width=1)
ax5.set_title('成交量', fontsize=12, fontweight='bold')
ax5.set_xlabel('日期')
ax5.set_ylabel('成交量')
ax5.grid(True, alpha=0.3, axis='y')

# 6. ATR指标
ax6 = axes[5]
atr = indicators['atr'].values
ax6.plot(df.index, atr, label='ATR', linewidth=2, color='orange')
ax6.set_title('平均真实波幅 (ATR)', fontsize=12, fontweight='bold')
ax6.set_xlabel('日期')
ax6.set_ylabel('ATR值')
ax6.legend()
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. 保存结果

In [ ]:
# 保存回测结果
import json
from datetime import datetime

# 创建结果目录
results_dir = '../../results'
os.makedirs(results_dir, exist_ok=True)

# 保存为JSON
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
result_file = os.path.join(results_dir, f'backtest_result_{timestamp}.json')

with open(result_file, 'w') as f:
    json.dump(result.to_dict(), f, indent=2, default=str)

print(f"回测结果已保存到: {result_file}")

# 保存权益曲线为CSV
equity_file = os.path.join(results_dir, f'equity_curve_{timestamp}.csv')
result.equity_curve.to_csv(equity_file)
print(f"权益曲线已保存到: {equity_file}")

## 9. 总结

In [ ]:
print("=" * 60)
print("回测总结")
print("=" * 60)

print(f"1. 策略表现: {result.total_return:.2%} 总收益 ({result.annual_return:.2%} 年化)")
print(f"2. 风险调整收益: 夏普比率 {result.sharpe_ratio:.2f}")
print(f"3. 风险控制: 最大回撤 {result.max_drawdown:.2%}")
print(f"4. 交易质量: 胜率 {result.win_rate:.2%} ({result.winning_trades}/{result.total_trades})")
print(f"5. 资金管理: 初始 ${result.initial_capital:,.2f} → 最终 ${result.final_capital:,.2f}")

if result.total_return > 0:
    print("\n✅ 策略表现积极")
    if result.sharpe_ratio > 1.0:
        print("✅ 夏普比率良好")
    if result.max_drawdown < 0.20:
        print("✅ 回撤控制良好")
    if result.win_rate > 0.50:
        print("✅ 胜率较高")
else:
    print("\n⚠️ 策略需要优化")
    
print("\n建议:")
print("1. 尝试不同的参数组合")
print("2. 添加更多技术指标过滤")
print("3. 考虑市场状态（牛市/熊市）")
print("4. 优化仓位管理和风险控制")